In [20]:
import pandas as pd
import numpy as np
import re

dfXcel = pd.read_excel('../RosterFiles/SEMDstaffingChart.xlsx', skiprows=1)
dfRoster = pd.DataFrame(columns=['Office', 'OrgCode', 'Name', 'PP',
                                 'Series', 'Grade', 'Status', 'FTE', 'Notes', 'eMail'])

C:\Users\wpulkown\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [21]:
dfXcel['Unnamed: 3'] = dfXcel['Division Immediate Office']

In [22]:
dfXcel['Unnamed: 3'] = np.where(~dfXcel['Division Immediate Office'].str.contains('Org', na=False),
                               np.nan, dfXcel['Division Immediate Office'])

In [23]:
dfXcel.loc[:,'Unnamed: 3'] = dfXcel.loc[:,'Unnamed: 3'].ffill()

In [24]:
dfOut = dfXcel[
    dfXcel['Division Immediate Office'].str.contains('([A-Z][a-zA-Z-]*)?([A-Z]\w+),\s([A-Z][^A-Z]\w+)\n?(\s[A-Z]\w*[.]?)?', na=False)]

C:\Users\wpulkown\AppData\Local\Temp/ipykernel_10528/611487203.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  dfXcel['Division Immediate Office'].str.contains('([A-Z][a-zA-Z-]*)?([A-Z]\w+),\s([A-Z][^A-Z]\w+)\n?(\s[A-Z]\w*[.]?)?', na=False)]


In [25]:
dfRoster['Name'] = dfOut['Division Immediate Office']
dfRoster['Office'] = dfOut['Unnamed: 3'] 
dfRoster['PP'] = dfOut['Unnamed: 4']
dfRoster['Series'] = dfOut['Unnamed: 5']
dfRoster['Grade'] = dfOut['Unnamed: 6']
dfRoster['Status'] = dfOut['Unnamed: 7']
dfRoster['FTE'] = dfOut['Unnamed: 8']
dfRoster['Notes'] = dfOut['Unnamed: 9']

In [26]:
def replace_commas(text):
    text = re.sub(', ', '.', text)
    return text

dfRoster['eMail'] = dfRoster['Name'].apply(lambda x: replace_commas(x))

In [27]:
dfRoster['eMail'] = dfRoster['eMail'].str.split().str[0]

In [28]:
dfRoster['eMail'] = dfRoster['eMail'] + '@epa.gov'

In [29]:
dfRecruitsOut =  dfXcel[
    dfXcel['Division Immediate Office']
    .str
    .contains(
        '(\d+) (Recruit)(?:[\w]+)?\s(?:[a-z\s]+)?(?:[()\dof\s]+)?([A-Z]\w+\s?){0,3}(\d{4})(\/\d{4})?(\/\d{4})?',
    na=False)]

C:\Users\wpulkown\AppData\Local\Temp/ipykernel_10528/3539632221.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  dfXcel['Division Immediate Office']


In [30]:
dfVacantOut = dfXcel[
    dfXcel['Division Immediate Office'].str.contains(
    'Vacant', na=False)]

In [31]:
dfRoster = pd.concat([dfRoster, dfRecruitsOut.rename(columns={'Division Immediate Office':'Name',
                                                  'Unnamed: 3':'Office', 'Unnamed: 4':'PP',
                                                   'Unnamed: 5': 'Series', 'Unnamed: 6': 'Grade',
                                                   'Unnamed: 7': 'Status', 'Unnamed: 8': 'FTE',
                                                   'Unnamed: 9': 'Notes'
                                                   })], ignore_index=True)

In [32]:
dfRoster = dfRoster.drop_duplicates(subset=['Name'], keep='last')

In [33]:
dfRoster = pd.concat([dfRoster, dfVacantOut.rename(columns={'Division Immediate Office':'Name',
                                                  'Unnamed: 3':'Office', 'Unnamed: 4':'PP',
                                                   'Unnamed: 5': 'Series', 'Unnamed: 6': 'Grade',
                                                   'Unnamed: 7': 'Status', 'Unnamed: 8': 'FTE',
                                                   'Unnamed: 9': 'Notes'
                                                   })], ignore_index=True)

In [34]:
dfRoster[['Office','OrgCode']] = dfRoster['Office'].str.split('-', 1, expand=True)

In [35]:
dfRoster['OrgCode'] = dfRoster['OrgCode'].str.split(n=1).str[0]

In [36]:
dfRoster = dfRoster.drop(columns=['Red - latest Staffing Updates   Green - Pathway Interns', 'Unnamed: 2'])

In [38]:
dfRoster.to_csv('../RosterFiles/cleanedRoster.csv')

And that's pretty much it.
TODO:
expand office acronyms